# RAW File Loading

In [ ]:
import os
import numpy as np

from metavision_core.event_io.raw_reader import RawReader
from metavision_core.event_io.py_reader import EventDatReader

## Opening a File

Opening a RAW or DAT file is as easy as creating a new `RawReader` or `EventDatReader` passing the path of the file.
Printing the created object will show some metadata about the reading status, such as the current index time and position.

Note how RAW files and DAT files contain different metadata.

In [ ]:
input_path_raw = "spinner.raw"
input_path_dat = "spinner.dat"
# if the file doesn't exist, it will be downloaded from Prophesee's public sample server 
from metavision_core.utils import get_sample

get_sample(input_path_raw, folder=".")
get_sample(input_path_dat, folder=".")

Here is an example of openning a RAW file.


In [ ]:
# open a file
record_raw = RawReader(input_path_raw)
print(record_raw)  # show some metadata

Here is an example of openning a DAT file.


In [ ]:
record_dat = EventDatReader(input_path_dat)
print(record_dat)

## Getting the Events

Now that the reader is created, we can get the events. This is done in the same way for both RAW and DAT files. Here we show an example using the RAW file, but the same can be done using the DAT file.

There are two ways to read events: 

* by number of events, using the `load_n_events()` function
* by time slices, using the `load_delta_t()` function


In [ ]:
events = record_raw.load_n_events(10)  # load the 10 next events
events_by_time = record_raw.load_delta_t(10000)  # load the next 10ms of data

print(events)

Events are stored in a numpy [structured array](https://numpy.org/doc/stable/user/basics.rec.html). This means you can conveniently access each attribute of the events ($x$,$y$, polarity $p$ and timestamp $t$) as a numpy array. It also allows to use native numpy operators such as [slicing](https://numpy.org/devdocs/user/basics.indexing.html?highlight=slice#combining-index-arrays-with-slices) or [boolean masking](https://numpy.org/devdocs/user/basics.indexing.html?highlight=slice#boolean-or-mask-index-arrays)

In [ ]:
# we can access different fields in this one dimensional array
print(events['t'])  # this shows only the timestamps of events
# for instance, to count the events of positive polarity you can do:
np.sum(events['p'] == 1)

In [ ]:
# let's randomly select some events
np.random.choice(events, len(events)//2)

## Iteration

The two functions `load_n_events()` and `load_delta_t()` can be used to read all events in a file using a `while` loop.

In [ ]:
sums = 0
#  load 10M events by batches of 50ms:
while not record_raw.is_done() and record_raw.current_event_index() < 1E7:
    # load the next 50 ms worth of events
    events = record_raw.load_delta_t(50000)
    sums += events.size
print("Event count: {}".format(sums))
print(record_raw)

You can restart from the beginning by calling the `reset` function. Note how all attributes are reset, and the event timestamps are the same as the first ones above.

In [ ]:
record_raw.reset()
print(record_raw)

## Visualization

We can visualize events using a function that will prepare an image from the numpy array of events.

Then we can display this image using standard visualization tools, such as [Matplotlib](<https://matplotlib.org/>).

In [ ]:

def viz_events(events, height, width):
    img = np.full((height, width, 3), 128, dtype=np.uint8)
    img[events['y'], events['x']] = 255 * events['p'][:, None]
    return img


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt  # graphic library, for plots

height, width = record_raw.get_size()

# load the next 50 ms worth of events
events = record_raw.load_delta_t(50000)
im = viz_events(events, height, width)

plt.imshow(im)
plt.tight_layout()